# Quantitative sensitivity analysis


## Generalized Sobol Indices


Here we show how to compute generalized Sobol indices on the **EOQ** model using the algorithm presented in Kucherenko et al. 2012. We import our model function from ``temfpy`` and use the Kucherenko indices function from ``econsa``. 

In [2]:
import matplotlib.pyplot as plt  # noqa: F401
import numpy as np

from temfpy.uncertainty_quantification import eoq_model

# TODO: Reactivate once Tim's PR is ready.
# from econsa.kucherenko import kucherenko_indices  # noqa: E265

The function ``kucherenko_indices`` expects the input function to be broadcastable over rows, that is, a row represents the input arguments for one evaluation. For sampling around the mean parameters we specify a diagonal covariance matrix, where the variances depend on the scaling of the mean. Since the variances of the parameters are unknown prior to our analysis we choose values such that the probability of sampling negative values is negligible. We do this since the **EOQ** model is not defined for negative parameters and the normal sampling does not naturally account for bounds.

In [2]:
def eoq_model_transposed(x):
    """EOQ Model but with variables stored in columns."""
    return eoq_model(x.T)

mean = np.array([1230, 0.0135, 2.15])
cov = np.diag([1, 0.000001, 0.01])

# indices = kucherenko_indices( # noqa: E265
#    func=eoq_model_transposed, # noqa: E265
#    sampling_mean=mean,  # noqa: E265
#    sampling_cov=cov,  # noqa: E265
#    n_draws=1_000_000,  # noqa: E265
#    sampling_scheme="sobol",  # noqa: E265
# )  # noqa: E265

Now we are ready to inspect the results.

In [3]:
# sobol_first = indices.loc[(slice(None), "first_order"), "value"].values  # noqa: E265
# sobol_total = indices.loc[(slice(None), "total"), "value"].values # noqa: E265

# x = np.arange(3)  # the label locations  # noqa: E265
# width = 0.35  # the width of the bars  # noqa: E265

# fig, ax = plt.subplots()  # noqa: E265
# rects1 = ax.bar(x - width / 2, sobol_first, width, label="First-order")  # noqa: E265
# rects2 = ax.bar(x + width / 2, sobol_total, width, label="Total")  # noqa: E265

# ax.set_ylim([0, 1])  # noqa: E265
# ax.legend()  # noqa: E265

# ax.set_xticks(x)  # noqa: E265
# ax.set_xticklabels(["$x_0$", "$x_1$", "$x_2$"])  # noqa: E265
# ax.legend();  # noqa: E265

In [4]:
# fig  # noqa: E265

## Shapley value

### Tutorial: A Linear Model with Three Inputs

#### The Execution

In [6]:
# import necessary packages and functions
import numpy as np
import chaospy as cp

from econsa.shapley import get_shapley
from econsa.shapley import _r_condmvn

Load all neccesary inputs for the model, you will need:
- a vector of mean estimates
- a covariance matrix
- the model you are conducting SA on
- the functions ``x_all`` and ``x_cond`` for conditional sampling. These functions depend on the distribution from which you are sampling from - for the purposes of this ilustration, we will sample from a multvariate normal distribution, but the functions can be tailored to needs.

In [69]:
# mean and covaraince matrix inputs
n_inputs = 3
mean = np.array([1230, 0.0135, 2.15])
cov = np.diag([1, 0.000001, 0.01])

In [93]:
# model for which senstivity analysis is being performed

def eoq_model_ndarray(x, r=0.1):
    """EOQ Model that accepts ndarray."""
    m = x[:,0]
    c = x[:,1]
    s = x[:,2]    
    return np.sqrt((24 * m * s) / (r * c))    

In [71]:
# functions for conditional sampling
def x_all(n):
    distribution = cp.MvNormal(mean, cov)
    return distribution.sample(n)

def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(subset_j, axis = 1)
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(n, mean = mean, cov = cov, dependent_ind = subset_j, given_ind = subsetj_conditional, x_given = xjc)

In [84]:
# estimate Shapley effects using the exact method
method = 'exact'
np.random.seed(1234)
n_perms = None
n_output = 10**4
n_outer = 10**3
n_inner = 10**2

exact_shapley = get_shapley(method, eoq_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

In [85]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
X1,0.007457,0.004145,-0.000668,0.015582
X2,0.713300,0.003929,0.705599,0.721001
X3,0.279243,0.004271,0.270872,0.287614


In [96]:
# estimate Shapley effects using the random method
method = 'random'
np.random.seed(1234)
n_perms = 20000
n_output = 10**4
n_outer = 1
n_inner = 3


random_shapley = get_shapley(method, eoq_model, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

In [97]:
random_shapley

,Shapley effects,std. errors,CI_min,CI_max
X1,0.007160,0.005201,-0.003035,0.017354
X2,0.712725,0.004876,0.703168,0.722281
X3,0.280116,0.004944,0.270426,0.289806


And there we have the resultant Shapley effects, in line with the chosen method of estimation. 

#### When do I use which method?
The `exact` method is good for use when the number of parameters is low, depending on the computational time it takes to estimate the model in question. If it is computationally inexpensive to estimate the model for which sensitivity analysis is required, then the `exact` method is always preferable, otherwise the `random` is recommended. A good way to proceed if one suspects that the computational time required to estimate the model is high, having a lot of parameters to conduct SA on is always to commence the exercise with a small number of parameters, e.g. 3, then get a benchmark of the Shapley effects using the `exact` method. Having done that, repeat the exercise using the `random` method on the same vector of parameters, calibrating the `n_perms` argument to the make sure that the results produced by the `random` method are the same as the `exact` one. Once this is complete, scale up the exercise using the `random` method, increasing the number of parameters to the desired parameter vector.

Now we plot the ranking of the Shapley values below.

In [ ]:
# plot for exact permutations method

In [98]:
# plot for random permutations method